In [5]:
import requests
from bs4 import BeautifulSoup
import time


In [2]:
# All games with metascore
metacritic_header = "https://www.metacritic.com/browse/game/"
year_range = "?releaseYearMin=2010&releaseYearMax=2024&page="

In [23]:
metacritic_header_game_page = "https://www.metacritic.com/game/"
def scrape_metacritic_game_page(game_name: str):
    game_url = game_name.replace(' ', '-').lower()
    url = metacritic_header_game_page + game_url
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    time.sleep(2)

    response = requests.get(url, headers=headers)
    print(f"Status Code: {response.status_code}")
    print(f"URL: {url}")

    soup = BeautifulSoup(response.content, 'html.parser')

    colors = ['red', 'yellow', 'green']
    score_template = "c-siteReviewScore u-flexbox-column u-flexbox-alignCenter u-flexbox-justifyCenter g-text-bold c-siteReviewScore_{color} g-color-white c-siteReviewScore_large"
    for c in colors:
        score_class = score_template.format(color=c)
        score_div = soup.find('div', {'class': score_class})
        if score_div:
            critic_score = score_div.find('span').text.strip()
            break

    
    platform_div = soup.find('div', {'class': 'c-gameDetails_Platforms u-flexbox u-flexbox-row'})
    platforms_list = list(map(lambda x: x.text.strip(), platform_div.find_all('li')))
    
    release_div = soup.find('div', {'class': 'c-gameDetails_ReleaseDate u-flexbox u-flexbox-row'})
    release_date = release_div.find_all('span')[1].text
    
    developer_div = soup.find('div', {'class': 'c-gameDetails_Developer u-flexbox u-flexbox-row'})
    developer_list = list(map(lambda x: x.text.strip(), developer_div.find_all('li')))
    
    publisher_div = soup.find('div', {'class': 'c-gameDetails_Distributor u-flexbox u-flexbox-row'})
    publisher = publisher_div.find_all('span')[1].text
    
    genres_div = soup.find('div', {'class': 'c-gameDetails_sectionContainer u-flexbox u-flexbox-row u-flexbox-alignBaseline'})
    genres_list = list(map(lambda x: x.text.strip(), genres_div.find_all('li')))

    data = {"Title": game_name,
            "Score": critic_score,
            "Platforms": platforms_list,
            "Release Date": release_date,
            "Developers": developer_list,
            "Publisher": publisher,
            "Genres": genres_list}
    return data

scrape_metacritic_game_page("the ")

Status Code: 200
URL: https://www.metacritic.com/game/elden-ring


AttributeError: 'NoneType' object has no attribute 'find'

In [8]:
def scrape_metacritic_page(page_no: str):
    assert page_no.isnumeric()
    url = metacritic_header + year_range + page_no
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    time.sleep(2)

    response = requests.get(url, headers=headers)
    print(f"Status Code: {response.status_code}")
    print(f"URL: {url}")

    soup = BeautifulSoup(response.content, 'html.parser')
    games = soup.find_all('div', {'class': 'c-finderProductCard c-finderProductCard-game'})
    if games is None:
        print("Game div class invalid")
        return None
    
    game_names = []
    for game in games:
        spans = game.find_all('span')
        span_texts = [span.text.strip() for span in spans if span.text.strip()]
        game_name = span_texts[1]
        if game_name:
            game_names.append(games)

    
    
    return game_names

In [9]:
t = scrape_metacritic_page("20")

Status Code: 200
URL: https://www.metacritic.com/browse/game/?releaseYearMin=2010&releaseYearMax=2024&page=20
24


In [10]:
print(*t, sep='\n')

['457.', 'Professor Layton and the Unwound Future', 'Sep 12, 2010', '•', 'Rated E10+', 'Rated', 'Professor Layton and his apprentice, Luke, receive a confounding letter - from Luke 10 years in the future. But the message inside is even more troubling: London of the future is in disarray, and the only person\r\nwho can help set things right is Professor Layton.', '86 Metascore', '86', 'Metascore']
['458.', 'Star Ocean: The Second Story R', 'Nov 2, 2023', '•', 'Rated T', 'Rated', "Space Date 366 - A Federation officer finds himself transported to a mystical planet. While searching for a way home, an encounter with a young girl draws him into a quest to save her people, just as an ancient prophecy foretold... \r\n\r\nChoose your path and witness an awakened destiny. \r\n\r\nThe highly acclaimed second instalment in the STAR OCEAN series, returns as STAR OCEAN THE SECOND STORY R. Brought to life with a unique 2.5D aesthetic, which fuses 2D pixel characters and 3D environments, this remake 